# 9.- Azure ML Deploy Training

In [31]:
import yaml

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml import Input, Output
from azure.ai.ml import command

In [32]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [33]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

data_asset_path = config["train"]["data_asset_path"]

In [34]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group_name,
    workspace_name
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [35]:
data_input = Input(
    path=config["train"]["data_asset_path"],
    type="uri_folder"
)

model_input = Input(
    path="../train/pretrained_models/yolo11n-seg.pt",
    type="uri_file"
)

In [36]:
output_path = "azureml://datastores/workspaceblobstore/paths/train_outputs/yolo_run_001"

output = Output(
    type="uri_folder"
)

In [37]:
command_job = """cp ${{ inputs.data }}/data.yaml /tmp/data.yaml && \
sed -i "s|path:.*$|path: ${{ inputs.data }}|" /tmp/data.yaml && \
python train.py"""

In [38]:
job = command(
    code="../src",
    command="python train.py",
    inputs={
        "data": data_input,
        "model": model_input,
    },
    outputs={
        "output_dir": output
    },
    environment_variables={ 
        "MODEL": model_input.path,
        "EPOCHS": "20",
        "BATCH": "4",
        "OUTPUT": output.path
    },
    environment="docker-context-example-walter@latest",
    compute="training-gpu-cluster",
    experiment_name="yolo11-segmentation-training",
    display_name="yolo11-segmentation-training"
)

In [ ]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)